Often it is useful to go beyound 2d and 1d nad store higher-dimension data - that is, data indexed by more than one or two keys. While there are `Panel` and `Panel4D`, it is more common to make user of *hierarchical indexing*. In this way, higher dimensional data can be compactly represented within the familiar 1d `Series` and 2d `DataFrame` objects.

In [1]:
import pandas as pd
import numpy as np

# A Multiply Indexed Series

Let's start by considering how we might represent 2d data within 1d `Series`.

## The bad way

Suppose you would like to track data about states from two different years. Useing the Pandas tool we've covered, how would you go about doing that?

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

However, it is very inconvenient to do anything else. For example, if you need to select all values from 2010, you would need to do something messy.

In [4]:
# How would you do it using list comprehension?
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

This produces the desired result, but not as clean and efficient

## The better way: Pandas MultiIndex

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

Notice that the `MultiIndex` contains multiple levels indexing - in this case, the state names and the years, as well as nultiple labels for each data point which encode these levels

If we re-index with this `MultiIndex`, we see the hierarchical representation of the data

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Here the first two columns of the `Series` representation show the multiple index values

In [7]:
pop.loc['California', :]

California  2000    33871648
            2010    37253956
dtype: int64

In [8]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## MultiIndex as extra dimension

sWe could have easily stored the same data using a simple `DataFrame` with index and column labels. The `unstack()` method will quickly convert a multiply indexed `Series` into a conventionally indexed `DataFrame`

In [9]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Naturally, the `stack()` method provides the opposite operation:

In [10]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

**Remark:** We can use it to represent data of three or more dimensinos in a `Series` or `DataFrame`. Each extra level in a multi-index represents an extra dimension of data. 

In [11]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In addition, all the ufunc and other functionality discussed work with hierarchical indices as well. Here we compute the fraction of people under 18 by year, given the above data:

In [12]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [13]:
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


This allows us to easily and quickly manipulate and explroe even high-dimensional data

# Methods of MultiIndex Creation

The most straightforward way to construct a multiply indexed `Series` or `DataFrame` is to simply pass a list of two or more index arrays to the constructor. For example:

In [14]:
df = pd.DataFrame(np.random.rand(4,2), 
                  index=[['a','a','b','b'], [1,2,1,2,]], 
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.717367  0.767156
  2  0.826411  0.039437
b 1  0.442441  0.068381
  2  0.929910  0.737250

The work of creating the `MultiIndex` is done in the background

Similarly, if you apss a dictionary with appropriate tuples as keys, Pandas will automatiacally recognize this and use a `MultiIndex` by default

In [15]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

## Explicit MultiIndex constructors

For more flexiblity in how the index is constructed, you can instead use the class method constructors available in the `pd.MultiIndex`. For example, as we did before, you can construct the `MultiIndex` from a simple list of arrays giving the index values within each level:

In [16]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b',], [1,2,1,2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

You can construct it from a list of tuples giving the multiple index values of each point:

In [17]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

You can even construct it from a Cartesian product of single indices:

In [18]:
pd.MultiIndex.from_product([['a', 'b'], [1,2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Similarly, you can construct the `MultiIndex` directly using its internal encoding by passing `levels` (a list of lists containing available index values for each level) and `labels` (a list of lists that reference these labels)

Any of these objects can be passed as the `index` argument when creating a `Series` or `DataFrame`, or be passed to the `reindex` method of an existing `Series` or `DataFrame`

## MultiIndex level names

Sometimes it is convenient to name the levels of the `MultiIndex`. This can be accomplished by passing the `names` argument to any of the above `MultiIndex` constructors, or by setting the `names` attribute of the index after the fact.

In [19]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## MultiIndex for columns

In [20]:
index = pd.MultiIndex.from_product([[2013,2014], [1,2]], 
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']], names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4,6),1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      23.0  36.9  20.0  35.9  46.0  36.3
     2      42.0  36.4  24.0  37.9  41.0  37.5
2014 1      37.0  37.8  55.0  37.5  24.0  34.4
     2      37.0  37.1  30.0  35.2  47.0  35.9

Here we see where the multi-indexing for both rows and columns can come in very handy. This is fundamentally 4d data, where the dimensions are the subject, the measurement type, the year, and the visit number. With this in place we can, for example, index the top-level column by the person's name and get a full `DataFrame` containing just that person's information.

In [21]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      20.0  35.9
     2      24.0  37.9
2014 1      55.0  37.5
     2      30.0  35.2

For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, cities, etc.) use of hierarchical rows and columns can be extremely convenient!

# Indexing and Slicing a MultiIndex

Indexing and slicing on a `MultiIndex` is designed to be intuitive, and it helps if you think about the indices as added dimensions. We will first look at indexing multiply indexed `Series`, and then multiply-indexed `DataFrame`

## Multiply Indexed Series

In [22]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

We can access single elements by indexing with multiple terms:

In [23]:
pop['California', 2000]

33871648

The `MultiIndex` also supports partial indexing, or indexing just one of the levels in the index. The result is another `Series`, with the lower-level indices maintained:

In [24]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

Partial slicing is available as well, as long as the `MultiIndex` is sorted

In [25]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:

In [26]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [27]:
# selection based on Boolean masks:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [28]:
# selection based on fancy indexing
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

## Multiply Indexed DataFrames

In [29]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      23.0  36.9  20.0  35.9  46.0  36.3
     2      42.0  36.4  24.0  37.9  41.0  37.5
2014 1      37.0  37.8  55.0  37.5  24.0  34.4
     2      37.0  37.1  30.0  35.2  47.0  35.9

In [30]:
# How to recover Guido's heart rate data?
health_data['Guido', 'HR']

year  visit
2013  1        20.0
      2        24.0
2014  1        55.0
      2        30.0
Name: (Guido, HR), dtype: float64

In [31]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      23.0  36.9
     2      42.0  36.4

These indexer provide an array-like view of the underlying 2d data, but each individual index in `loc` or `iloc` can be passed a tuple of multiple indices. For example:

In [32]:
health_data.loc[:, ('Bob','HR')]

year  visit
2013  1        23.0
      2        42.0
2014  1        37.0
      2        37.0
Name: (Bob, HR), dtype: float64

Working with slices within these index tuples is not especially convenient; trying to create a slice within a tuple will lead to a syntax error:

In [33]:
health_data.loc[(:,1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-33-4167e8501d5a>, line 1)

You could get aroudn this by building the desired slice explicitly using Python's built-in `slice()` function, but a better way in this context is to use an `IndexSlice` object, which Pandas provides for precisely this situation. For example:

In [34]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,23.0,20.0,46.0
2014,1,37.0,55.0,24.0


# Rearranging Multi-Indices

One of the way to working with multiply indexed data is knowing how to effectively transform the data. There are a number of operations that will preserve all the infomation in the dataset, but rearrange it for the purposees of various computations. We saw a brief example of this in the `Stack()` and `unstack()` method, but there are many more ways to finely control the rearrangement of data between hierarchical indices and columns, and we will explore them here.

## Sorted and unsorted indices

**Remark:** Many of the `MultiIndex` slicing operations will fail if the index is not sorted.

We will start by creating some simple multiply indexed data where the indices are not *lexographically sorted*

In [35]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1,2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names=['char', 'int']
data

char  int
a     1      0.730946
      2      0.671151
c     1      0.869519
      2      0.722069
b     1      0.966842
      2      0.188767
dtype: float64

If we try to take a partial slice of this index, it will result in an error:

In [36]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


For various reasons, partial slices and other similar operations require the levels in the `MultiIndex` to be in sorted order. Pandas provides a number of convenience routines to perform this type of sorting; examples are the `sort_index()` and `sortlevel()` methods of the DataFrame. We'll use the simplest, `sort_index()`, here:

In [37]:
data = data.sort_index()
data

char  int
a     1      0.730946
      2      0.671151
b     1      0.966842
      2      0.188767
c     1      0.869519
      2      0.722069
dtype: float64

In [38]:
# With index sorted in this way, partial slicing will work as expected:
data['a':'b']

char  int
a     1      0.730946
      2      0.671151
b     1      0.966842
      2      0.188767
dtype: float64

## Stacking and unstacking indices

As we saw briefly before, it is possible to convert a dataset from a stacked multi-index to a simple 2d representation, optionally specifying the level to use:

In [39]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [41]:
pop.unstack()

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [42]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [43]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


The opposite of `unstack()` is `stack()`, which here can be used to recover the original series:

In [50]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

## Index setting and resetting

Another way to rearrange hierarchical data is to turn the index labels into columns; this can be accomplished with the `reset_index` method. Calling this on the population dictionary will result in a `DataFrame` with a statte and year column holding the information that was formerly in the index. For clarity, we can optionally specify the name of the data for the column representation:

In [55]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [53]:
pop_flat = pop.reset_index()
pop_flat

,state,year,0
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


Often when working with data in the real world, the raw input data looks like this and it's useful to build a `MultiIndex` from the column values. This can be done with the `set_index` method of the `DataFrame`, which returns a multiply indexed `DataFrame`

In [56]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

# Data Aggregations on Multi-Indices

For hierarchically indexed data, methods such as `mean()`, `sum()`, and `max()` can be passed a level parameter that controls which subset of the data the aggregate is computed on.


In [58]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      23.0  36.9  20.0  35.9  46.0  36.3
     2      42.0  36.4  24.0  37.9  41.0  37.5
2014 1      37.0  37.8  55.0  37.5  24.0  34.4
     2      37.0  37.1  30.0  35.2  47.0  35.9

In [65]:
# agerage out the measurements in the two visits each year
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     32.5  36.65  22.0  36.90  43.5  36.90
2014     37.0  37.45  42.5  36.35  35.5  35.15

In [66]:
data_mean.index

Int64Index([2013, 2014], dtype='int64', name='year')

By further making use of the `axis` keyword, we can take the mean among levels on the columns as well:


In [74]:
data_mean.mean(axis=1, level='type')


type,HR,Temp
year,,
2013,32.666667,36.816667
2014,38.333333,36.316667


In [69]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,32.666667,36.816667
2014,38.333333,36.316667


This syntax is a shortcut to the `GroupBy` functionality, which we will discuss in Aggregation and Group. 